## Intermediate tools of NumPy for scientific computing

### About

- This tutorial builds on [`numpy_basics.ipynb`](https://github.com/jhparkyb/NumAnalNotes_Pub/blob/master/computations/numpy_basics.ipynb). 
- `numpy_intermediate.ipynb` collects tools geared towards a more detailed needs while `numpy_basics.ipynb` is a quick start toolbox for an Introductory Numerical Analysis course. 

Author: Jea-Hyun Park



### License

This work is licensed under [Creative Commons Attribution-ShareAlike 4.0 International](https://creativecommons.org/licenses/by-sa/4.0/)
Part of the content of this notebook is borrowed from various reference, which are linked for more details. Thanks to all great contributors.

#### `nonzero` (returns fancy indexing)

- `numpy.nonzero` returns a fancy indexing type arrays where nonzero elements are located.
- `numpy.nonzero` can be combined with masking since `True` is treated as `1` (nonzero) and `False` as `0`.
- It can be used as a method: `arr.nonzero()` is equivalent to `numpy.nonzero(arr)`.

In [13]:
import numpy as np

x = np.array([[3, 0, 0], [0, 4, 0], [5, 6, 0]])
ind = np.nonzero(x)
x_nonzero = x[ind]

print("original array:\n", x)
print("indices of non-zero elements:\n", ind)
print("non-zero elements:\n", x_nonzero)


original array:
 [[3 0 0]
 [0 4 0]
 [5 6 0]]
indices of non-zero elements:
 (array([0, 1, 2, 2]), array([0, 1, 0, 1]))
non-zero elements:
 [3 4 5 6]


In [16]:
a = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
mask = a > 5
ind_nonzero = np.nonzero(mask)

print("array:\n", a)
print("mask:\n", mask)
print("indices of non-zero elements:\n", ind_nonzero)

# `np.nonzero` can be used similarly to masking
print("a[np.nonzero(a > 5)]:\n",a[np.nonzero(a > 5)])
print("a[a > 5]:\n", a[a > 5])

array:
 [[1 2 3]
 [4 5 6]
 [7 8 9]]
mask:
 [[False False False]
 [False False  True]
 [ True  True  True]]
indices of non-zero elements:
 (array([1, 2, 2, 2]), array([2, 0, 1, 2]))
a[np.nonzero(a > 5)]:
 [6 7 8 9]
a[a > 5]:
 [6 7 8 9]


#### Array computation rules

1. Operations between multiple array objects are first checked for proper shape match (*broadcasting*).
1. All operations and functions are element-wise.
2. Reduction operations (`numpy.mean`, `numpy.std`, `numpy.sum`, `numpy.prod`, `numpy.max`, `numpy.min`, etc) apply to the whole array, unless an `axis` is specified.
    - For example, `np.sum(a, axis = 0)` means 'sum entries of array `a` **along** axis 0 (the outer most index according to *row-major* rule)'. Therefore, the axis feeded will go away.
    - `axis = -1` always sum along the inner most index, i.e. it aggregates each column vectors. (See [Row-major high-dimensional arrays](#row-major-high-dimensional-arrays))
3. Missing values (`nan`, stands for 'not a number') propagate unless explicitely ignored (`numpy.nanmean`, `numpy.nansum`, etc).

In [ ]:
import numpy as np

# rule 4: nan propagate unless using 
a = np.array([np.nan, 1, 3, 5])
print(a)
print(np.mean(a)) # take average including nan's
print(np.nanmean(a)) # take average excluding nan's

#### Reshape and transpose


##### Reshape and transpose behave differently

In [4]:
import numpy as np

# reshape vs transpose
a_orig = np.arange(5*3, dtype=np.float64).reshape((5,3))

a_reshaped = a_orig.reshape(3,5)
a_transposed = a_orig.T

print("a original (5,3)\n", a_orig)
print("a reshaped to (3,5)\n", a_reshaped)
print("a transposed to (3,5)\n", a_transposed)

a original (5,3)
 [[ 0.  1.  2.]
 [ 3.  4.  5.]
 [ 6.  7.  8.]
 [ 9. 10. 11.]
 [12. 13. 14.]]
a reshaped to (3,5)
 [[ 0.  1.  2.  3.  4.]
 [ 5.  6.  7.  8.  9.]
 [10. 11. 12. 13. 14.]]
a transposed to (3,5)
 [[ 0.  3.  6.  9. 12.]
 [ 1.  4.  7. 10. 13.]
 [ 2.  5.  8. 11. 14.]]


##### Reshaped data access: linear index

> **Summary**
>
> `arr_ori[ind1]==arr_reshaped[ind2]` iff linear index of `ind1` = `ind2`


- The original array and the new array has the same data iff their linear indices are the same.
- How to get multidimensional index from a linear index: `multi_dim_ind = np.unravel_index(lin_index, shape)`.
- How to get linear index from a multidimensional index: `lin_ind = np.ravel_multi_index(multi_int, shape)`.
  - These can be used to conduct mutiple such tasks by passing `dim=` parameter. (See the documentation for detail.)
  - These can do the *column-major* version by passing `order=` parameter. (See the documentation for detail.)

In [12]:
import numpy as np

# create an array with random entries, and its reshape
arr_ori = np.random.rand(2, 3, 5, 7)
arr_reshaped = arr_ori.reshape((3, 5, 2, 7))

# Two indices for each array: These are chosen to be linear index (or flattened index) = 122 (less than 2*3*5*7 = 210)
fan_ind = (1,0,2,3)
ind_reshaped = (1,3,1,3)

# print the values of the two indices
print("original:", fan_ind, arr_ori[fan_ind])
print("reshaped:", ind_reshaped, arr_reshaped[ind_reshaped])

# print the linear index of the two indices
print("linear index of original array:", np.ravel_multi_index(fan_ind, (2,3,5,7)))
print("linear index of reshaped array:", np.ravel_multi_index(ind_reshaped, (3,5,2,7)))


original: (1, 0, 2, 3) 0.17226201704659516
reshaped: (1, 3, 1, 3) 0.17226201704659516
linear index of original array: 122
linear index of reshaped array: 122


##### Transpose data access: permutation

- 1D array: `transpose` does NOTHING.
- 2D array: `transpose` behaves the same way as matrix transpose.
- High-diminsional array: Use permutation (usually a `tuple` of indices) to specify transpositions.
  - Supposing we have named `perm` our permutation as in the example below, `perm` represents the *resulting* dimensions in terms of the origianl dimensions: `perm[i]`-th dimension of the new array = i-th dimension of the original array. (See the summary below.)

> **Summary**
> 
> Given `new = old.transpose(perm)`, we have
> 
> - `new.shape[i] == old.shape[perm[i]] for i in range(old.ndim)` 
> - `old[ind_old] == new[ind_new]` iff `np.all([ind_new[i]==ind_old[perm[i]] for i in range(old.ndim)])` 

In [34]:
import numpy as np

# create an array with random entries, and its reshape
arr_ori = np.random.rand(2, 3, 5, 7)

# permutation of dimensions for transpose
# new.shape[i] = old.shape[perm[i]] for i = 0, 1, 2, 3
perm = (2, 0, 3, 1)
arr_transposed = arr_ori.transpose(perm)

# check the shape of the two arrays
print([arr_transposed.shape[i] == arr_ori.shape[perm[i]] for i in range(arr_ori.ndim)])

# Two indices using a permutation of dimensions
fan_ind = (1,0,-1,5)
ind_transposed = (-1,1,5,0)

# print the values of the two indices
print("original:", fan_ind, arr_ori[fan_ind])
print("reshaped:", ind_transposed, arr_transposed[ind_transposed])
print("Entry equality condition:", np.all([ind_transposed[i]==fan_ind[perm[i]] for i in range(arr_ori.ndim)]))

[True, True, True, True]
[True, True, True, True]
original: (1, 0, -1, 5) 0.02619612663819626
reshaped: (-1, 1, 5, 0) 0.02619612663819626
Entry equality condition: True


We can make the index conversion as a function to reuse conveniently.

In [36]:
def permute_index(ind_ori, perm):
    """Permute the index according to the permutation of dimensions

    INPUTS:
    -------
    ind_ori: tuple
        original index
    perm: tuple
        permutation of dimensions
    OUTPUTS:
    --------
    ind_permuted: tuple
        permuted index
    """
    ind_permuted = tuple([ind_ori[perm[i]] for i in range(len(ind_ori))])
    return ind_permuted

ind_transposed = permute_index(fan_ind, perm) # using a function

# check the index matching of the two arrays
print([ind_transposed[i] == fan_ind[perm[i]] for i in range(arr_ori.ndim)])

# print the values of the two indices
print("original:", fan_ind, arr_ori[fan_ind])
print("reshaped:", ind_transposed, arr_transposed[ind_transposed])
print("Entry equality condition:", np.all([ind_transposed[i]==fan_ind[perm[i]] for i in range(arr_ori.ndim)]))

[True, True, True, True]
original: (1, 0, -1, 5) 0.02619612663819626
reshaped: (-1, 1, 5, 0) 0.02619612663819626
Entry equality condition: True


**Inversion of transpose**

- Apply the inverse permutation.
- NumPy offers `argsort` for this purpose (see the example below).
- More explicitly, the inverse permutation is obtained by creating an array with its `perm[i]`-th entry being `i` back as its original had: `inv_perm[perm[i]]=i` for `i range in len(perm)`. A vectorized version reads: `inv_perm = np.zeros(len(perm), dtype=np.int64); inv_perm[perm] = arange(len(perm));`.
- (Wrong way) Applying the same permutation twice results in something different since a permutation squared is not the identity permutation except for some special cases (with multipication being interpreted as compositions).

> **Summary** (inversion of transposition)
> 
> Given `perm` is used for transposition,
>
> - `arr_ori == arr_transposed.transpose(inv_perm)` iff `inv_perm == np.argsort(perm)`

In [49]:
# inverting transposition using `argsort`, manual, vectorized manual
inv_perm = np.argsort(perm)
inv_perm_manual = (1,3,0,2)
inv_perm_manual2 = np.zeros(len(perm), dtype=np.int64)
inv_perm_manual2[np.array(perm)] = np.arange(len(perm))

print("original shape:", arr_ori.shape)
print("transposed shape:", arr_transposed.shape)
print("(WRONG inversion) transposing twice using the same permutation:", arr_transposed.transpose(perm).shape)
print("back to origianl (argsort):", arr_transposed.transpose(np.argsort(perm)).shape)
print("back to origianl (manual):", arr_transposed.transpose(inv_perm_manual).shape)

print("Inverse permutation condition check:", np.all(inv_perm_manual == np.argsort(perm)))
print("Inverse permutation condition check:", np.all(inv_perm_manual == inv_perm_manual2))

original shape: (2, 3, 5, 7)
transposed shape: (5, 2, 7, 3)
(WRONG inversion) transposing twice using the same permutation: (7, 5, 3, 2)
back to origianl (argsort): (2, 3, 5, 7)
back to origianl (manual): (2, 3, 5, 7)
Inverse permutation condition check: True
Inverse permutation condition check: True


#### `copysign`

- We can give sign (`+` or `-`) we want using `copysign` function. 
- Basic syntax
  - `numpy.copysign(magnitude_arr, sign_arr)` returns an array with the magnitude of `magnitude_arr` and the sign of `sign_arr`.
- Native Python `math` module offers the same funcationality.
- Treatment of 0.
  - Python version: The documentation says that if the plaform support negative zero separately, then `-0.0` is considered `negative`.
  - NumPy version: `numpy` documentation is not explicit about this. But apparently, it offers `0.0` and `-0.0` separately. And they are treated the same way as the Python version.

See the [`copysign` documentation (NumPy)](https://numpy.org/doc/stable/reference/generated/numpy.copysign.html) and [`copysign` documentation (native Python)](https://docs.python.org/3/library/math.html#math.copysign) for more details.

In [1]:
import numpy as np

# magnitude and sign arrays
magnitude_arr = np.array([1, 2, 3, 4, 5])
sign_arr = np.arange(5) - 2

# use `copysign` to create a signed array
signed_arr = np.copysign(magnitude_arr, sign_arr)

# print the arrays
print("magnitude_arr:", magnitude_arr)
print("sign_arr:", sign_arr)
print("signed_arr:", signed_arr)


magnitude_arr: [1 2 3 4 5]
sign_arr: [-2 -1  0  1  2]
signed_arr: [-1. -2.  3.  4.  5.]


In [8]:
# examples showing treatment of 0's

print(1/np.array([-0.]))
print(1/np.array([0]))

print(1/np.copysign(0, 1))
print(1/np.copysign(0, -1))

[-inf]
[inf]
inf
-inf


/tmp/ipykernel_26840/833596975.py:3: RuntimeWarning: divide by zero encountered in true_divide
  print(1/np.array([-0.]))
/tmp/ipykernel_26840/833596975.py:4: RuntimeWarning: divide by zero encountered in true_divide
  print(1/np.array([0]))
/tmp/ipykernel_26840/833596975.py:6: RuntimeWarning: divide by zero encountered in double_scalars
  print(1/np.copysign(0, 1))
/tmp/ipykernel_26840/833596975.py:7: RuntimeWarning: divide by zero encountered in double_scalars
  print(1/np.copysign(0, -1))


### Appendix

#### PyTorch and NumPy: Differences

PyTorch, which is a popular tool for implementing neural networks, is also used as a scientific computing. The developers of PyTorch have emulated NumPy. But there are some different conventions for the same or similar functionalities. The primary reason for the change of names appears to make things more suggestive of what they really give.

Setting: `arr` is a numpy array and `tsr` is a torch Tensor.

| | NumPy | PyTorch |
|---|---|---|
| main object | `array` (`numpy.array`) | `tensor` (`torch.tensor`) |
| total dimensionality | `arr.ndim` | `arr.ndim` |
| organization of array | `arr.shape` | `tsr.size()`|
| total number of entries | `arr.size` | `tsr.numel()`|

In [8]:
import torch as tc
import numpy as np

arr = np.arange(5)
tsr = tc.from_numpy(arr)

print("numpy arr: ", arr)
print("  - type: ", type(arr))
print("  - ndim: ", arr.ndim)
print("  - shape: ", arr.shape)
print("  - size: ", arr.size)

print("torch tsr: ", tsr)
print("  - type: ", type(tsr))
print("  - ndim: ", tsr.ndim)
print("  - shape: ", tsr.shape)
print("  - size(): ", tsr.size())
print("  - size: ", tsr.size)
print("  - nelm: ", tsr.numel())


numpy arr:  [0 1 2 3 4]
  - type:  <class 'numpy.ndarray'>
  - shape:  (5,)
  - size:  5
  - ndim:  1
torch tsr:  tensor([0, 1, 2, 3, 4])
  - type:  <class 'torch.Tensor'>
  - shape:  torch.Size([5])
  - size():  torch.Size([5])
  - size:  <built-in method size of Tensor object at 0x7fc2213a5950>
  -nelm:  5
  - ndim:  1
